In [411]:
import pandas as pd
import numpy as np
import os
import unicodedata
import re
from collections import defaultdict

# Define a manual replacement dictionary
replacement_map = {
    "ø": "o",
    "ł": "l",
    "đ": "d",
    "ŋ": "n",
    "ß": "ss",
    "æ": "ae",
    "œ": "oe",
    "ð": "d",
    "þ": "th",
    "ĸ": "k"
}

KICKER_FILE='D:\\DevOps\\python_work\\venv\\demoenv\\resources\\output_k.json'
TF_FILE='D:\\DevOps\\python_work\\venv\\demoenv\\resources\\output_tf.json'
FIFA_FILE='D:\\DevOps\\python_work\\venv\\demoenv\\resources\\players_fifa.json'

def load_json_file_into_dataframe(file:str):
    try:
        # Check if file exists
        if not os.path.exists(file):
            raise FileNotFoundError(f"File '{file}' not found.")

        # Load JSON file into a DataFrame
        print(f"Loadinf {file}...")
        return pd.read_json(file)
    except FileNotFoundError as fnf_error:
            print(fnf_error)
            return []
        
def drop_columns_from_dataframe(data:pd.DataFrame, columns:list):
    return data.drop(columns, axis='columns')

def dataframe_reports(data_list:list):
    [(dataframe_report(data)) for data in data_list]    

def transform_to_datetime(data:pd.DataFrame, key:str):
    data[key] = pd.to_datetime(data[key], errors="coerce")
    return data

def dataframe_report(data:pd.DataFrame):
    print(f"Data frame has {len(data)} entries.")
    print(data.info())

# Function to normalize names
def normalize_name(first_name, last_name):
    # Convert None to empty string
    first_name = first_name or ""
    last_name = last_name or ""
    
    # Convert to lowercase
    full_name = f"{first_name} {last_name}".strip().lower()
    
    # Remove accents and special characters
    full_name = ''.join(
        c for c in unicodedata.normalize("NFKD", full_name) if unicodedata.category(c) != "Mn"
    )

    # Normalize to NFKD (splits special characters from accents)
    normalized = unicodedata.normalize("NFKD", full_name)
    
    # Convert accents to base ASCII equivalent
    ascii_text = "".join(c for c in normalized if not unicodedata.combining(c))
    
    # Apply manual replacements for specific cases
    for special_char, replacement in replacement_map.items():
        ascii_text = ascii_text.replace(special_char, replacement)
    
    # Replace spaces with underscores
    full_name = re.sub(r"\s+", "_", full_name)
    
    return full_name

# Check for near matches (only among unmatched)
def is_similar(name1, name2):
    #print(f"{name1} :: {name2}")
    """Check if two names are similar by length difference and common substrings."""
    if name1 is None or name2 is None:
        return False
    if not isinstance(name1, str) or not isinstance(name2, str):
        return False
    if abs(len(name1) - len(name2)) > 10:  # Allow minor length variations
        return False
    return name1 in name2 or name2 in name1  # Simple containment check

def match_on_additional_atributes(merged_df:pd.DataFrame):
    unmatched_df1 = merged_df[merged_df["_merge"] == "left_only"]
    unmatched_df2 = merged_df[merged_df["_merge"] == "right_only"]
    
    # Secondary matching on similar names but same date_of_birth
    index_list = []
    df_um = pd.DataFrame(columns=merged_df.columns)
    for index1, row1 in unmatched_df1.iterrows():
        for index2, row2 in unmatched_df2.iterrows():
            if row1["dateOfBirth"] == row2["dateOfBirth"]:
                # Check if names are similar
                norm1 = row1["normalized_name"]
                norm2 = row2["normalized_name"]
                if (norm1 in norm2 or norm2 in norm1
                    or is_similar(row1["firstName_x"], row2["firstName_y"])
                    or is_similar(row1["lastName_x"], row2["lastName_y"])
                    ):  # Basic substring match
                    
                    s1 = pd.Series(row1)
                    s2 = pd.Series(row2)
                    if np.isnan(df_um.index.max()):
                        df_um.loc[unmatched_df1.index.max() + 1] = s1.combine_first(s2)
                    else:
                        df_um.loc[df_um.index.max() + 1] = s1.combine_first(s2)

                    index_list.append(index1)
                    index_list.append(index2)
                    #print(row1)
    print(f"index list> {index_list}")
    print(f"merged_df count> {len(merged_df)}")
    merged_df.drop(index_list, inplace=True)
    print(f"merged_df count> {len(merged_df)}")
    
    #merged_df.iloc[index_list]["_merge"] = 'merged'
    df_um["_merge"]='both_'
    return df_um

def remove_players_from_wrong_competition(data:pd.DataFrame, key:str):
    # Count occurrences of each club
    club_counts = data[key].value_counts()
    clubs_filtered=club_counts[club_counts<5].index.tolist()
    data_filtered = data[~data[key].isin(clubs_filtered)]
    print(f"From {len(data)} are {len(data_filtered)} left > {len(data)-len(data_filtered)} removed.")
    return data_filtered

def group_players_by_team(data:pd.DataFrame, team_key:str):
    keys=data.columns.to_list()
    # Grouping by 'team'
    team_dict = (data.groupby(keys[2]).
                 agg({keys[0]: list, keys[1]: list}).
                 #apply(lambda x: {"count": len(x["firstName"]),"players": list(zip(x["firstName"], x["lastName"]))}, axis=1).
                 apply(lambda x: {"count": len(x[keys[0]])}, axis=1).
                 to_dict())

    return team_dict

def merge_dataframes(df1:pd.DataFrame, df2:pd.DataFrame, merge_on:list, how="outer"):
    # Merge DataFrames
    merged_df = pd.merge(df1, df2, how=how, left_on=merge_on, right_on=merge_on, indicator=True)

    # Count matches and non-matches
    matched_entries = merged_df["_merge"].value_counts().get("both", 0)
    unmatched_entries = merged_df["_merge"].value_counts().get("left_only", 0) + merged_df["_merge"].value_counts().get("right_only", 0)

    print(f"Total merged (found) entries: {matched_entries}")
    print(f"Total without hits: {unmatched_entries}")

    return merged_df 

def report_unmachted_items(df:pd.DataFrame):
    # Show only the unmatched entries
    unmatched_df = df[df["_merge"] != "both"][["normalized_name", "_merge", "nn_x", "nn_y"]]
    print("\nEntries without a match:\n", unmatched_df)

kicker=load_json_file_into_dataframe(KICKER_FILE)
tf=load_json_file_into_dataframe(TF_FILE)
fifa=load_json_file_into_dataframe(FIFA_FILE)

kicker=drop_columns_from_dataframe(kicker, ["geburtsname", "namensverlauf"])
tf=drop_columns_from_dataframe(tf, ["Outfitter", "nativeName"])

# transform to date type 
kicker = transform_to_datetime(kicker, "dateOfBirth")
tf = transform_to_datetime(tf, "dateOfBirth")
fifa = transform_to_datetime(fifa, "birthdate")

# Apply name normalization
kicker["normalized_name"] = kicker.apply(lambda row: normalize_name(row["firstName"], row["lastName"]), axis=1)
tf["normalized_name"] = tf.apply(lambda row: normalize_name(row["first_name"], row["last_name"]), axis=1)
kicker["nn"] = kicker.apply(lambda row: normalize_name(row["firstName"], row["lastName"]), axis=1)
tf["nn"] = tf.apply(lambda row: normalize_name(row["first_name"], row["last_name"]), axis=1)
fifa["normalized_name"] = fifa.apply(lambda row: normalize_name(row["firstName"], row["lastName"]), axis=1)
fifa["nn"] = fifa.apply(lambda row: normalize_name(row["firstName"], row["lastName"]), axis=1)

kicker=remove_players_from_wrong_competition(kicker, "currentClub")
tf=remove_players_from_wrong_competition(tf, "currentClub")
fifa=remove_players_from_wrong_competition(fifa, "team")

tf=tf.rename(columns={"first_name":"firstName", "last_name":"lastName"})
fifa=fifa.rename(columns={"birthdate":"dateOfBirth"})

kicker=kicker.drop_duplicates(subset=["normalized_name", "dateOfBirth"])

kicker["source"] = "kicker"
tf["source"] = "tf"
fifa["source"] = "fifa"

dataframe_reports([kicker, tf, fifa])


Loadinf D:\DevOps\python_work\venv\demoenv\resources\output_k.json...
Loadinf D:\DevOps\python_work\venv\demoenv\resources\output_tf.json...
Loadinf D:\DevOps\python_work\venv\demoenv\resources\players_fifa.json...
From 551 are 516 left > 35 removed.
From 508 are 498 left > 10 removed.
From 483 are 483 left > 0 removed.
Data frame has 505 entries.
<class 'pandas.core.frame.DataFrame'>
Index: 505 entries, 0 to 547
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               505 non-null    object        
 1   firstName        505 non-null    object        
 2   lastName         505 non-null    object        
 3   currentClub      505 non-null    object        
 4   dateOfBirth      505 non-null    datetime64[ns]
 5   height           505 non-null    int64         
 6   weight           505 non-null    int64         
 7   position         505 non-null    object        
 8   nations        

C:\Users\Der David\AppData\Local\Temp\ipykernel_7588\3731244769.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[key] = pd.to_datetime(data[key], errors="coerce")


In [63]:
merged_df=merge_dataframes(kicker, tf, ["normalized_name","dateOfBirth"])
print(merged_df.index.is_unique)
report_unmachted_items(merged_df)
print(merged_df.info())

print(f"merged_df BEFORE> {len(merged_df)}")
df_um=match_on_additional_atributes(merged_df)
print(f"merged_df AFTER> {len(merged_df)}")

print(merged_df["_merge"].value_counts())
print(df_um["_merge"].value_counts())
df_combined = pd.concat([merged_df, df_um], ignore_index=True)

# Count matches and non-matches
matched_entries = df_combined["_merge"].value_counts().get("both", 0) + df_combined["_merge"].value_counts().get("both_", 0) 
unmatched_entries = df_combined["_merge"].value_counts().get("left_only", 0) + df_combined["_merge"].value_counts().get("right_only", 0)

print("\n")
print(f"Total players (found) entries: {len(df_combined)}")
print(f"Total merged (found) entries: {matched_entries}")
print(f"Total without hits: {unmatched_entries}")
report_unmachted_items(df_combined)

Total merged (found) entries: 483
Total without hits: 37
True

Entries without a match:
              normalized_name      _merge                     nn_x  \
49            artem_stepanov   left_only           artem_stepanov   
66   eric_junior_dina_ebimbe   left_only  eric_junior_dina_ebimbe   
68             oscar_hojlund   left_only            oscar_hojlund   
85                 maxim_dal   left_only                maxim_dal   
94             hyunseok_hong   left_only            hyunseok_hong   
144              viggo_gebel   left_only              viggo_gebel   
147     joyeux_masanka_bungi   left_only     joyeux_masanka_bungi   
152                     xavi   left_only                     xavi   
158               faik_sakar   left_only               faik_sakar   
174            bennit_broger   left_only            bennit_broger   
183          mohammed_amoura   left_only          mohammed_amoura   
217    tiago_pereira_cardoso   left_only    tiago_pereira_cardoso   
223           

In [316]:
merged_df=merge_dataframes(kicker, tf, ["normalized_name","dateOfBirth"])
report_unmachted_items(merged_df)
df_um=match_on_additional_atributes(merged_df)
print(df_um.head())
# Combine them
df_combined = pd.concat([merged_df, df_um], ignore_index=True)
df_combined.info()

# Merge all three DataFrames on first_name, last_name, and date_of_birth
triple_df = kicker.merge(tf, on=["firstName", "lastName", "dateOfBirth"], how="outer")\
               .merge(fifa, on=["firstName", "lastName", "dateOfBirth"], how="outer", indicator=True)

triple_df[["normalized_name", "dateOfBirth", "_merge"]]

# Count matches and non-matches
matched_entries = triple_df["_merge"].value_counts().get("both", 0)
unmatched_entries = triple_df["_merge"].value_counts().get("left_only", 0) + triple_df["_merge"].value_counts().get("right_only", 0)

print(f"Total merged (found) entries: {matched_entries}")
print(f"Total without hits: {unmatched_entries}")

triple_df.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\players_all.csv", encoding='utf-8', index=False)

Total merged (found) entries: 494
Total without hits: 37

Entries without a match:
              normalized_name dateOfBirth      _merge
50            artem_stepanov  2007-08-10   left_only
67   eric_junior_dina_ebimbe  2000-11-21   left_only
69             oscar_hojlund  2005-01-04   left_only
86                 maxim_dal  2006-01-26   left_only
95             hyunseok_hong  1999-06-16   left_only
148              viggo_gebel  2007-11-22   left_only
151     joyeux_masanka_bungi  2007-01-24   left_only
156                     xavi  2003-04-21   left_only
162               faik_sakar  2008-01-14   left_only
179            bennit_broger  2006-07-01   left_only
188          mohammed_amoura  2000-05-09   left_only
224    tiago_pereira_cardoso  2006-04-07   left_only
230            julian_chabot  1998-02-12   left_only
266           almugera_kabar  2006-02-06   left_only
354            leo_ostigaard  1999-11-28   left_only
355          finn_ole_becker  2000-06-08   left_only
372         ere

In [114]:
print(merged_df["id_y"])
merged_df["id_y"]=merged_df["id_y"].apply(lambda x: str(x) if pd.notna(x) else "")

#merged_df["firstName_y"]=merged_df["firstName_y"].map(str)
print(merged_df.info())

#print(merged_df["id_y"].to_list())

0       17259
1      468539
2       40680
3      607720
4      475413
        ...  
515    334999
516    808664
517    618898
518    303219
519    746910
Name: id_y, Length: 520, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_x             520 non-null    object        
 1   firstName_x      505 non-null    object        
 2   lastName_x       505 non-null    object        
 3   currentClub_x    505 non-null    object        
 4   dateOfBirth      520 non-null    datetime64[ns]
 5   height_x         505 non-null    float64       
 6   weight           505 non-null    float64       
 7   position_x       505 non-null    object        
 8   nations          505 non-null    object        
 9   age              505 non-null    float64       
 10  normalized_name  520 non-null    object        
 11  nn_x    

In [185]:
kicker_teams=group_players_by_team(kicker[["firstName", "lastName", "currentClub"]], "currentClub")
tf_teams=group_players_by_team(tf[["first_name", "last_name", "currentClub"]], "currentClub")
fifa_teams=group_players_by_team(fifa[["firstName", "lastName", "team"]], "team")
print(kicker_teams)
print(tf_teams)
print(fifa_teams)

{'1. FC Heidenheim': {'count': 30}, '1. FC Union Berlin': {'count': 27}, '1. FSV Mainz 05': {'count': 25}, 'Bayer 04 Leverkusen': {'count': 26}, 'Bayern München': {'count': 27}, 'Bor. Mönchengladbach': {'count': 26}, 'Borussia Dortmund': {'count': 29}, 'Eintracht Frankfurt': {'count': 27}, 'FC Augsburg': {'count': 29}, 'FC St. Pauli': {'count': 36}, 'Holstein Kiel': {'count': 29}, 'RB Leipzig': {'count': 29}, 'SC Freiburg': {'count': 27}, 'TSG Hoffenheim': {'count': 32}, 'VfB Stuttgart': {'count': 33}, 'VfL Bochum': {'count': 27}, 'VfL Wolfsburg': {'count': 30}, 'Werder Bremen': {'count': 27}}
{'1.FC Heidenheim 1846': {'count': 28}, '1.FC Union Berlin': {'count': 26}, '1.FSV Mainz 05': {'count': 23}, 'Bayer 04 Leverkusen': {'count': 26}, 'Bayern Munich': {'count': 26}, 'Borussia Dortmund': {'count': 28}, 'Borussia Mönchengladbach': {'count': 25}, 'Eintracht Frankfurt': {'count': 27}, 'FC Augsburg': {'count': 27}, 'FC St. Pauli': {'count': 34}, 'Holstein Kiel': {'count': 29}, 'RB Leipzi

In [334]:
il = [1,5,8]
m=df_um.tail()
m.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\merged_df.csv", encoding='utf-8', index=False)

In [329]:
df_ktf=df_combined[["normalized_name","dateOfBirth"]]
# transform to date type 
df_ktf = transform_to_datetime(df_ktf, "dateOfBirth")

merged_df_ktf=merge_dataframes(df_ktf, fifa, ["normalized_name","dateOfBirth"])
report_unmachted_items(merged_df_ktf)
merged_df_ktf.info()
#df_um_ktff=match_on_additional_atributes(merged_df_ktf)


Total merged (found) entries: 419
Total without hits: 174

Entries without a match:
                    normalized_name dateOfBirth      _merge
3                      jonas_urbig  2003-08-08   left_only
10                     min-jae_kim  1996-11-15   left_only
16                   joao_palhinha  1995-07-09   left_only
30                          arthur  2003-03-17   left_only
33              alejandro_grimaldo  1995-09-20   left_only
..                             ...         ...         ...
588                   ayman_aourir  2004-10-06  right_only
589                    andu_kelati  2002-08-13  right_only
590  kaua_morais_vieira_dos_santos  2003-04-11  right_only
591                  mats_pannewig  2004-08-28  right_only
592                   yannic_stein  2004-09-17  right_only

[174 rows x 3 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           ---

C:\Users\Der David\AppData\Local\Temp\ipykernel_39896\1337660496.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[key] = pd.to_datetime(data[key], errors="coerce")


In [420]:
print(kicker[["normalized_name", "firstName", "lastName", "dateOfBirth"]].head(10))
print(tf[["normalized_name", "firstName", "lastName", "dateOfBirth"]].head(10))
print(fifa[["normalized_name", "firstName", "lastName", "dateOfBirth"]].head(10))

names=set(kicker["normalized_name"]) & set(tf["normalized_name"]) & set(fifa["normalized_name"])
pd.DataFrame({"normalized_name": list(names)})

k=kicker[["normalized_name", "firstName", "lastName", "dateOfBirth"]]
t=tf[["normalized_name", "firstName", "lastName", "dateOfBirth"]]
f=fifa[["normalized_name", "firstName", "lastName", "dateOfBirth"]]

k.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\k.csv", encoding='utf-8', index=False)
t.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\t.csv", encoding='utf-8', index=False)
f.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\f.csv", encoding='utf-8', index=False)

     normalized_name firstName   lastName dateOfBirth
0       manuel_neuer    Manuel      Neuer  1986-03-27
1      daniel_peretz    Daniel     Peretz  2000-07-10
2       sven_ulreich      Sven    Ulreich  1988-08-03
3        jonas_urbig     Jonas      Urbig  2003-08-08
4         sacha_boey     Sacha       Boey  2000-09-13
5     tarek_buchmann     Tarek   Buchmann  2005-02-28
6    alphonso_davies  Alphonso     Davies  2000-11-02
7          eric_dier      Eric       Dier  1994-01-15
8  raphael_guerreiro   Raphael  Guerreiro  1993-12-22
9         hiroki_ito    Hiroki        Ito  1999-05-12
    normalized_name firstName    lastName dateOfBirth
0      kevin_muller     Kevin      Müller  1991-03-15
1      frank_feller     Frank      Feller  2004-01-07
2      vitus_eicher     Vitus      Eicher  1990-11-05
3   paul_tschernuth      Paul  Tschernuth  2002-01-20
4   benedikt_gimber  Benedikt      Gimber  1997-02-19
5    patrick_mainka   Patrick      Mainka  1994-11-06
6    tim_siersleben       Ti

In [406]:
import pandas as pd
import re

def normalize_name(name):
    if not name:
        return ""
    # Convert to lowercase
    name = name.lower()
    # Replace special characters with ASCII equivalent
    name = ''.join(c if not unicodedata.combining(c) else '' for c in unicodedata.normalize('NFKD', name))
    # Remove non-alphanumeric characters (except spaces)
    name = re.sub(r'[^a-z0-9 ]', '', name)
    # Merge spaces and strip
    return "_".join(name.split())

print(normalize_name("Raphaël Guerreiro"))
print(normalize_name("Oscar Højlund"))
print(normalize_name("Adam Dźwigałła"))
print(normalize_name("Adam Moussâ"))


''.join(c if not unicodedata.combining(c) else '' for c in unicodedata.normalize('NFKD', "Højlund"))

raphael_guerreiro
oscar_hjlund
adam_dzwigaa
adam_moussa


'Højlund'

In [443]:
import pandas as pd

# Sample DataFrame
data1 = {
    "firstName": ["Florian", "Harry", "Jeremie"],
    "lastName": ["Wirtz", "Kane", "Frimpong"],
    "team": ["Leverkusen", "Bayern Munich", "Leverkusen"],
    "normalized_name": ["florian_wirtz", "harry_kane", "jeremie_frimpong"]
}
data2 = {
    "firstName": ["Florian", "Harry", "Jeremie"],
    "lastName": ["Wirtz", "Kane", "Frimpong"],
    "team": ["Leverkusen", "Bayern Munich", "Leverkusen"],
    "normalized_name": ["florian_wirtz", "harry_kane", "jeremie_frimpong"]
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

merged_df = pd.merge(df1, df2, how='left', left_on="lastName", right_on="lastName", indicator=True)
merged_df

,firstName_x,lastName,team_x,normalized_name_x,firstName_y,team_y,normalized_name_y,_merge
0,Florian,Wirtz,Leverkusen,florian_wirtz,Florian,Leverkusen,florian_wirtz,both
1,Harry,Kane,Bayern Munich,harry_kane,Harry,Bayern Munich,harry_kane,both
2,Jeremie,Frimpong,Leverkusen,jeremie_frimpong,Jeremie,Leverkusen,jeremie_frimpong,both


In [467]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_x             505 non-null    object
 1   firstName_x      505 non-null    object
 2   lastName_x       505 non-null    object
 3   currentClub_x    505 non-null    object
 4   dateOfBirth      508 non-null    object
 5   height_x         505 non-null    object
 6   weight           505 non-null    object
 7   position_x       505 non-null    object
 8   nations          505 non-null    object
 9   age              505 non-null    object
 10  normalized_name  508 non-null    object
 11  nn_x             505 non-null    object
 12  id_y             498 non-null    object
 13  slug             498 non-null    object
 14  lastName_y       498 non-null    object
 15  firstName_y      498 non-null    object
 16  placeOfBirth     493 non-null    object
 17  height_y         496 non-null    ob

In [478]:
df_combined=df_combined.rename(columns={"firstName_x":"firstName", "lastName_x":"lastName", "_merge":"_oldmerge"})
# transform to date type 
df_combined = transform_to_datetime(df_combined, "dateOfBirth")

fully_merged_df=merge_dataframes(df_combined, fifa, ["normalized_name","dateOfBirth"])
report_unmachted_items(fully_merged_df)
print(fully_merged_df.info())

fully_merged_df.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\fully_merged_df.csv", encoding='utf-8', index=False)

fully_merged.drop_duplicates(subset=["normalized_name", "dateOfBirth"])
fully_merged=fully_merged_df.reset_index()
df_um_fully=match_on_additional_atributes(fully_merged_df)
print(fully_merged_df["_merge"].value_counts())
print(df_um_fully["_merge"].value_counts())
df_combined_fully = pd.concat([fully_merged_df, df_um_fully], ignore_index=True)



Total merged (found) entries: 409
Total without hits: 173

Entries without a match:
                    normalized_name      _merge                nn_x  \
3                      jonas_urbig   left_only         jonas_urbig   
10                     min-jae_kim   left_only         min-jae_kim   
16                   joao_palhinha   left_only       joao_palhinha   
29                          arthur   left_only              arthur   
32              alejandro_grimaldo   left_only  alejandro_grimaldo   
..                             ...         ...                 ...   
577                   ayman_aourir  right_only                 NaN   
578                    andu_kelati  right_only                 NaN   
579  kaua_morais_vieira_dos_santos  right_only                 NaN   
580                  mats_pannewig  right_only                 NaN   
581                   yannic_stein  right_only                 NaN   

                   nn_y  
3           jonas_urbig  
10          min-jae_ki

ValueError: cannot reindex on an axis with duplicate labels

In [557]:
name_mapping = {
    "id_x":"id",
    "firstName_x":"firstName",
    "lastName_x":"lastName",
    "currentClub_x":"currentClub",
    "_merge": "_1stmerge",
    "firstName_y":"firstName_yy",
    "lastName_y":"lastName_yy",
    "id_y":"id_yy",
    "height_y":"height_yy",
}

merged_df=merged_df.rename(columns=name_mapping)
#merged_df.info()

fully_merged_df=merge_dataframes(merged_df, fifa, ["normalized_name","dateOfBirth"])
report_unmachted_items(fully_merged_df)
print(fully_merged_df.info())

fully_merged_df.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\fully_merged_df.csv", encoding='utf-8', index=False)
#fully_merged_df=merged_df.rename(columns=name_mapping)
fully_merged_df.info()

Total merged (found) entries: 405
Total without hits: 169

Entries without a match:
                    normalized_name      _merge                nn_x  \
3                      jonas_urbig   left_only         jonas_urbig   
10                     min-jae_kim   left_only         min-jae_kim   
16                   joao_palhinha   left_only       joao_palhinha   
29                          arthur   left_only              arthur   
32              alejandro_grimaldo   left_only  alejandro_grimaldo   
..                             ...         ...                 ...   
569                   ayman_aourir  right_only                 NaN   
570                    andu_kelati  right_only                 NaN   
571  kaua_morais_vieira_dos_santos  right_only                 NaN   
572                  mats_pannewig  right_only                 NaN   
573                   yannic_stein  right_only                 NaN   

                   nn_y  
3           jonas_urbig  
10          min-jae_ki

In [560]:

um = fully_merged_df[(fully_merged_df["_merge"] == "left_only") | (fully_merged_df["_merge"] == "right_only")]
um.info()

df_um_triple=match_on_additional_atributes(fully_merged_df)
print(fully_merged_df["_merge"].value_counts())
print(df_um_triple["_merge"].value_counts())
df_um_triple.info()
# unmatched_df1 = merged_df[merged_df["_merge"] == "left_only"]
# unmatched_df2 = merged_df[merged_df["_merge"] == "right_only"]

<class 'pandas.core.frame.DataFrame'>
Index: 122 entries, 3 to 573
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_x             65 non-null     object        
 1   firstName_x      65 non-null     object        
 2   lastName_x       65 non-null     object        
 3   currentClub      65 non-null     object        
 4   dateOfBirth      122 non-null    datetime64[ns]
 5   height_x         65 non-null     float64       
 6   weight_x         65 non-null     float64       
 7   position_x       65 non-null     object        
 8   nations          65 non-null     object        
 9   age              65 non-null     float64       
 10  normalized_name  122 non-null    object        
 11  nn_x             65 non-null     object        
 12  source_x         65 non-null     object        
 13  id_yy            60 non-null     float64       
 14  slug             60 non-null     object        

In [564]:
df_triple_combined = pd.concat([fully_merged_df, df_um_triple], ignore_index=True)

# Count matches and non-matches
matched_entries = df_triple_combined["_merge"].value_counts().get("both", 0) + df_triple_combined["_merge"].value_counts().get("both_", 0) 
unmatched_entries = df_triple_combined["_merge"].value_counts().get("left_only", 0) + df_triple_combined["_merge"].value_counts().get("right_only", 0)

print("\n")
print(f"Total players (found) entries: {len(df_triple_combined)}")
print(f"Total merged (found) entries: {matched_entries}")
print(f"Total without hits: {unmatched_entries}")
report_unmachted_items(df_triple_combined)

df_triple_combined.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\df_triple_combined.csv", encoding='utf-8', index=False)
merged_df.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\merged_df.csv", encoding='utf-8', index=False)





Total players (found) entries: 527
Total merged (found) entries: 405
Total without hits: 122

Entries without a match:
                    normalized_name      _merge              nn_x  \
3                      jonas_urbig   left_only       jonas_urbig   
27                          arthur   left_only            arthur   
30                   mario_hermoso   left_only     mario_hermoso   
36                emiliano_buendia   left_only  emiliano_buendia   
43                     alejo_sarco   left_only       alejo_sarco   
..                             ...         ...               ...   
522                   ayman_aourir  right_only               NaN   
523                    andu_kelati  right_only               NaN   
524  kaua_morais_vieira_dos_santos  right_only               NaN   
525                  mats_pannewig  right_only               NaN   
526                   yannic_stein  right_only               NaN   

                 nn_y  
3         jonas_urbig  
27           

C:\Users\Der David\AppData\Local\Temp\ipykernel_39896\4254397674.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_triple_combined = pd.concat([fully_merged_df, df_um_triple], ignore_index=True)
C:\Users\Der David\AppData\Local\Temp\ipykernel_39896\4254397674.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_triple_combined = pd.concat([fully_merged_df, df_um_triple], ignore_index=True)
C:\Users\Der David\AppData\Local\Temp\ipykernel_39896\4254397674.py:1: FutureWarning: The behavior o

In [588]:

merged_df=merge_dataframes(kicker, tf, ["normalized_name","dateOfBirth"])
merged_df.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\merged_df.csv", encoding='utf-8', index=False)
df_um=match_on_additional_atributes(merged_df)
df_um.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\df_um.csv", encoding='utf-8', index=False)

df_combined = pd.concat([merged_df, df_um], ignore_index=True)
df_combined.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\df_combined.csv", encoding='utf-8', index=False)

name_mapping = {
    "id_x":"id",
    "firstName_x":"firstName",
    "lastName_x":"lastName",
    "currentClub_x":"currentClub",
    "_merge": "_1stmerge",
    "firstName_y":"firstName_yy",
    "lastName_y":"lastName_yy",
    "id_y":"id_yy",
    "height_y":"height_yy",
}

df_combined=df_combined.rename(columns=name_mapping)
df_combined = transform_to_datetime(df_combined, "dateOfBirth")
df_combined.info()

fully_merged_df=merge_dataframes(df_combined, fifa, ["normalized_name","dateOfBirth"])
report_unmachted_items(fully_merged_df)
print(fully_merged_df.info())

fully_merged_df.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\fully_merged_df.csv", encoding='utf-8', index=False)
#fully_merged_df=merged_df.rename(columns=name_mapping)
fully_merged_df.info()

df_um_triple=match_on_additional_atributes(fully_merged_df)
print(fully_merged_df["_merge"].value_counts())
print(df_um_triple["_merge"].value_counts())
df_um_triple.info()

df_um_triple

df_triple_combined = pd.concat([fully_merged_df, df_um_triple], ignore_index=True)

df_triple_combined.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\df_triple_combined.csv", encoding='utf-8', index=False)

Total merged (found) entries: 483
Total without hits: 37
index list> [66, 511, 68, 510, 94, 507, 152, 513, 183, 519, 223, 518, 343, 514, 344, 515, 361, 517, 467, 512, 489, 505, 497, 506]
merged_df count> 520
merged_df count> 496
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               505 non-null    object        
 1   firstName        505 non-null    object        
 2   lastName         505 non-null    object        
 3   currentClub      505 non-null    object        
 4   dateOfBirth      508 non-null    datetime64[ns]
 5   height_x         505 non-null    object        
 6   weight           505 non-null    object        
 7   position_x       505 non-null    object        
 8   nations          505 non-null    object        
 9   age              505 non-null    object        
 10  normalized_name  508 non-

In [596]:
club_counts = kicker2["currentClub"].value_counts()
print(club_counts)

currentClub
FC St. Pauli            36
VfB Stuttgart           33
TSG Hoffenheim          32
1. FC Heidenheim        30
VfL Wolfsburg           30
RB Leipzig              29
Holstein Kiel           29
Borussia Dortmund       29
FC Augsburg             29
VfL Bochum              27
1. FC Union Berlin      27
Werder Bremen           27
Bayern München          27
SC Freiburg             27
Eintracht Frankfurt     27
Bor. Mönchengladbach    26
Bayer 04 Leverkusen     26
1. FSV Mainz 05         25
TSG Hoffenheim II        3
1. FC Kaiserslautern     3
Real Valladolid          2
RB Salzburg              2
Borussia Dortmund II     2
FC St. Gallen            1
Stade Reims              1
Debreceni VSC            1
Tottenham Hotspur        1
Galatasaray SK           1
Fortuna Düsseldorf       1
FC Lausanne-Sport        1
Lazio Rom                1
Plymouth Argyle          1
AEL Limassol             1
AC Florenz               1
FC Sevilla               1
1. FC Nürnberg           1
Royal Antwerp FC

In [606]:
        df1 = pd.DataFrame({
            "firstName": ["Florian", "Harry", "Jeremie"],
            "lastName": ["Wirtz", "Kane", "Frimpong"],
            "team": ["Leverkusen", "Bayern Munich", "Leverkusen"],
            "normalized_name": ["florian_wirtz", "harry_kane", "jeremie_frimpong"]
        })
        df2 = pd.DataFrame({
            "lastName": ["Wirtz", "Kane", "Müller", "Frimpong"],
            "age": [22, 31, 33, 27],
            "team": ["Leverkusen", "Bayern Munich", "Bayern Munich", "Leverkusen"],
            "normalized_name": ["florian_wirtz", "harry_kane", "thomas_muller", "jeremie_frimpong"]
        })
print(df1)
print(df2)
merge_dataframes(df1, df2, ["normalized_name"])

  firstName  lastName           team   normalized_name
0   Florian     Wirtz     Leverkusen     florian_wirtz
1     Harry      Kane  Bayern Munich        harry_kane
2   Jeremie  Frimpong     Leverkusen  jeremie_frimpong
   lastName  age           team   normalized_name
0     Wirtz   22     Leverkusen     florian_wirtz
1      Kane   31  Bayern Munich        harry_kane
2    Müller   33  Bayern Munich     thomas_muller
3  Frimpong   27     Leverkusen  jeremie_frimpong
Total merged (found) entries: 3
Total without hits: 1


,firstName,lastName_x,team_x,normalized_name,lastName_y,age,team_y,_merge
0,Florian,Wirtz,Leverkusen,florian_wirtz,Wirtz,22,Leverkusen,both
1,Harry,Kane,Bayern Munich,harry_kane,Kane,31,Bayern Munich,both
2,Jeremie,Frimpong,Leverkusen,jeremie_frimpong,Frimpong,27,Leverkusen,both
3,NaN,NaN,NaN,thomas_muller,Müller,33,Bayern Munich,right_only


In [120]:
tf["id"] = tf["id"].apply(lambda x: str(x) if pd.notna(x) else "")

merged_df=merge_dataframes(kicker, tf, ["normalized_name","dateOfBirth"])
df_left=merged_df[merged_df["_merge"]=="left_only"][["id_x", "dateOfBirth", "normalized_name", "firstName_x","lastName_x"]]
df_right=merged_df[merged_df["_merge"]=="right_only"][["id_y", "dateOfBirth","normalized_name", "firstName_y","lastName_y"]]
print(df_left.head(12))
print(df_right.head(10))






Total merged (found) entries: 483
Total without hits: 37
                        id_x dateOfBirth          normalized_name  \
49            artem-stepanov  2007-08-10           artem_stepanov   
66   eric-junior-dina-ebimbe  2000-11-21  eric_junior_dina_ebimbe   
68            oscar-hoejlund  2005-01-04            oscar_hojlund   
85                 maxim-dal  2006-01-26                maxim_dal   
94             hyunseok-hong  1999-06-16            hyunseok_hong   
144              viggo-gebel  2007-11-22              viggo_gebel   
147     joyeux-masanka-bungi  2007-01-24     joyeux_masanka_bungi   
152              xavi-simons  2003-04-21                     xavi   
158               faik-sakar  2008-01-14               faik_sakar   
174           bennit-broeger  2006-07-01            bennit_broger   
183  mohammed-elamine-amoura  2000-05-09          mohammed_amoura   
217    tiago-pereira-cardoso  2006-04-07    tiago_pereira_cardoso   

     firstName_x       lastName_x  
49       

In [121]:
import pandas as pd
import itertools

# Rename columns in df1 and df2 to match the final format
df_left = df_left.rename(columns={"id_x": "id", "firstName_x": "firstName", "lastName_x": "lastName"})
df_right = df_right.rename(columns={"id_y": "id", "firstName_y": "firstName", "lastName_y": "lastName"})
# Concatenate the two DataFrames
final_df = pd.concat([df_left, df_right], ignore_index=True)
# Reorder columns for consistency
final_df = final_df[["id", "firstName", "lastName", "normalized_name", "dateOfBirth"]]
final_df["id"]=final_df["id"].astype(str)
print(final_df.dtypes)
print(final_df.tail())

# Group by 'dateOfBirth' and filter groups with more than two entries
grouped_df = final_df.groupby("dateOfBirth").filter(lambda x: len(x) > 1)

print(grouped_df.columns)
grouped_df[["normalized_name","dateOfBirth", "firstName", "lastName"]].sort_values(by=['dateOfBirth'], ascending=False).head(25)
print(f"grouped_df :: {grouped_df.head()} >>")
print()

def is_similar(name1, name2):
    print(f"{name1} :: {name2}")
    """Check if two names are similar by length difference and common substrings."""
    if not name1 or not name2:
        return False
    if not isinstance(name1, str) or not isinstance(name2, str):
        return False
    if abs(len(name1) - len(name2)) > 10:  # Allow minor length variations
        return False
    return name1 in name2 or name2 in name1  # Simple containment check

print(grouped_df.head())
print()

# Iterate over each group
for date, group in grouped_df.groupby("dateOfBirth"):
    print(f"\nChecking group with dateOfBirth: {date}")
    
    # Compare names within the group
    for (idx1, row1), (idx2, row2) in itertools.combinations(group.iterrows(), 2):
        if is_similar(row1["lastName"], row2["lastName"]):
            print(f"  Similar last names: {row1['lastName']} ~ {row2['lastName']} {row1['id']}::{(row2['id'])}")
            break
        if is_similar(row1["firstName"], row2["firstName"]):
            print(f"  Similar first names: {row1['firstName']} ~ {row2['firstName']} {row1['id']}::{row1['id']}")
            break
        if is_similar(row1["normalized_name"], row2["normalized_name"]):
            print(f"  Similar normalized names: {row1['normalized_name']} ~ {row2['normalized_name']} {row1['id']}::{row1['id']}")
            break


id                         object
firstName                  object
lastName                   object
normalized_name            object
dateOfBirth        datetime64[ns]
dtype: object
        id firstName  lastName   normalized_name dateOfBirth
32  334999      Finn    Becker       finn_becker  2000-06-08
33  808664   Florian  Micheler  florian_micheler  2005-05-17
34  618898   Erencan  Yardımcı  erencan_yardımcı  2002-02-04
35  303219      Jeff    Chabot       jeff_chabot  1998-02-12
36  746910   Mohamed    Amoura    mohamed_amoura  2000-05-09
Index(['id', 'firstName', 'lastName', 'normalized_name', 'dateOfBirth'], dtype='object')
grouped_df ::                          id    firstName     lastName  \
1   eric-junior-dina-ebimbe  Eric Junior  Dina Ebimbe   
2            oscar-hoejlund        Oscar      Höjlund   
4             hyunseok-hong     Hyunseok         Hong   
7               xavi-simons                      Xavi   
10  mohammed-elamine-amoura     Mohammed       Amoura   

    

In [122]:
# Ensure the column is in datetime format
final_df["dateOfBirth"] = pd.to_datetime(final_df["dateOfBirth"], errors="coerce")

final_df.info()
final_df["dateOfBirth"]=final_df["dateOfBirth"].dt.strftime("%Y-%m-%d")
final_df["id"]=final_df["id"].astype(str)
final_df.to_json(orient="records", force_ascii=False)
#print(f'{final_df["dateOfBirth"]} :: {final_df["dateOfBirth"].dt.strftime("%Y-%m-%d")}')
#print(f'{final_df["dateOfBirth"].dt.strftime("%Y-%m-%d")}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               37 non-null     object        
 1   firstName        37 non-null     object        
 2   lastName         37 non-null     object        
 3   normalized_name  37 non-null     object        
 4   dateOfBirth      37 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 1.6+ KB


'[{"id":"artem-stepanov","firstName":"Artem","lastName":"Stepanov","normalized_name":"artem_stepanov","dateOfBirth":"2007-08-10"},{"id":"eric-junior-dina-ebimbe","firstName":"Eric Junior","lastName":"Dina Ebimbe","normalized_name":"eric_junior_dina_ebimbe","dateOfBirth":"2000-11-21"},{"id":"oscar-hoejlund","firstName":"Oscar","lastName":"Höjlund","normalized_name":"oscar_hojlund","dateOfBirth":"2005-01-04"},{"id":"maxim-dal","firstName":"Maxim","lastName":"Dal","normalized_name":"maxim_dal","dateOfBirth":"2006-01-26"},{"id":"hyunseok-hong","firstName":"Hyunseok","lastName":"Hong","normalized_name":"hyunseok_hong","dateOfBirth":"1999-06-16"},{"id":"viggo-gebel","firstName":"Viggo","lastName":"Gebel","normalized_name":"viggo_gebel","dateOfBirth":"2007-11-22"},{"id":"joyeux-masanka-bungi","firstName":"Joyeux","lastName":"Masanka Bungi","normalized_name":"joyeux_masanka_bungi","dateOfBirth":"2007-01-24"},{"id":"xavi-simons","firstName":"","lastName":"Xavi","normalized_name":"xavi","dateOfB

In [147]:
print(merged_df[merged_df["id_x"] == "haktab-omar-traore"])
print(merged_df[merged_df["id_y"] == "388294"])
t=merged_df.loc[[489,505]]
#t.to_json(orient="records", force_ascii=False)
#({'haktab-omar-traore'}, {'388294'})
#df_right=merged_df[merged_df["_merge"]=="right_only"][["id_y", "dateOfBirth","normalized_name", "firstName_y","lastName_y"]]

  

# Set _merge column to "_both"
new_merged_df["_merge"] = "_both"

new_merged_df

                   id_x  firstName_x lastName_x     currentClub_x dateOfBirth  \
489  haktab-omar-traore  Omar Haktab     Traoré  1. FC Heidenheim  1998-02-04   

     height_x  weight position_x              nations   age  ... citizenship  \
489     187.0    78.0     Abwehr  [Deutschland, Togo]  27.0  ...         NaN   

    position_y foot currentClub_y joined expires marketValue nn_y  source_y  \
489        NaN  NaN           NaN    NaN     NaN         NaN  NaN       NaN   

        _merge  
489  left_only  

[1 rows x 29 columns]
    id_x firstName_x lastName_x currentClub_x dateOfBirth  height_x  weight  \
505  NaN         NaN        NaN           NaN  1998-02-04       NaN     NaN   

    position_x nations  age  ...      citizenship             position_y  \
505        NaN     NaN  NaN  ...  [Germany, Togo]  Defender - Right-Back   

      foot         currentClub_y       joined       expires  \
505  right  1.FC Heidenheim 1846  Jul 1, 2023  Jun 30, 2026   

                     

,dateOfBirth,id_x,firstName_x,lastName_x,currentClub_x,height_x,weight,position_x,nations,age,...,citizenship,position_y,foot,currentClub_y,joined,expires,marketValue,nn_y,source_y,_merge
0,1998-02-04,haktab-omar-traore,Omar Haktab,Traoré,1. FC Heidenheim,187.0,78.0,Abwehr,"[Deutschland, Togo]",27.0,...,"[Germany, Togo]",Defender - Right-Back,right,1.FC Heidenheim 1846,"Jul 1, 2023","Jun 30, 2026","{'current': 4000000.0, 'highest': 4000000.0}",omar_traore,tf,_both


In [193]:
# Combine all dataframes
df_all = pd.concat([kicker, tf, fifa], ignore_index=True)
#print(df_all.head())
# Step 3: Add "count" column per group
df_all["count"] = df_all["dateOfBirth"].map(df_all["dateOfBirth"].value_counts())
#print(df_all.head())
# Add count_similarities (count of identical normalized_name within a group)
df_all["count_similar"] = df_all.groupby(["dateOfBirth", "normalized_name"])["normalized_name"].transform("count")
# Remove rows where count and count_similar are both 3
#df_all = df_all[~((df_all['count'] == 3) & (df_all['count_similar'] == 3))]
#df_all.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\df_all.csv", encoding='utf-8', index=False)
# Group by 'dateOfBirth'
grouped = df_all.groupby("dateOfBirth")
#print(grouped.head())
# Check if 'id' is unique in the concatenated DataFrame
unique_ids = df_all['normalized_name'].is_unique
print(f"Are 'id' values unique? {unique_ids}")

# Iterate through groups and print selected columns
for date, group in grouped:
    print(group[['normalized_name', 'dateOfBirth', 'firstName', 'lastName', 'source', 'count', 'count_similar']].to_string(index=False))
    print("\n")

df_all.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\df_all_wo_similar.csv", encoding='utf-8', index=False)

Are 'id' values unique? False
normalized_name dateOfBirth firstName lastName source  count  count_similar
anthony_losilla  1986-03-10   Anthony  Losilla kicker      3              3
anthony_losilla  1986-03-10   Anthony  Losilla     tf      3              3
anthony_losilla  1986-03-10   Anthony  Losilla   fifa      3              3


normalized_name dateOfBirth firstName lastName source  count  count_similar
   manuel_neuer  1986-03-27    Manuel    Neuer kicker      3              3
   manuel_neuer  1986-03-27    Manuel    Neuer     tf      3              3
   manuel_neuer  1986-03-27    Manuel    Neuer   fifa      3              3


  normalized_name dateOfBirth firstName   lastName source  count  count_similar
norman_theuerkauf  1987-01-24    Norman Theuerkauf kicker      3              3
norman_theuerkauf  1987-01-24    Norman Theuerkauf     tf      3              3
norman_theuerkauf  1987-01-24    Norman Theuerkauf   fifa      3              3


    normalized_name dateOfBirth   fi

In [292]:
# search terms is list of column names
# terms df holds the grouped players based on dob
def extract(search_terms:list, terms:pd.DataFrame):
    #print(f"{search_terms} :: {terms.to_dict()}")
    #print()
    
    matched = []
    for i, row in terms.iterrows():
        print(f'Row> {i}, {row} :: {type(row)} >>')
        for search_i, term in enumerate(search_terms):
            if row.iloc[search_i] == "" or search_terms[search_i] == "":
                matched.append((row.iloc[search_i], False, i))
                break
            print(f'Search> {row.iloc[search_i]} :: {search_terms[search_i]}')
            if (row.iloc[search_i] == search_terms[search_i]) or (row.iloc[search_i] in search_terms[search_i]) or (search_terms[search_i] in row.iloc[search_i]):
                matched.append((row.iloc[search_i], True, i))
                break
            else:
                matched.append((row.iloc[search_i], False, i))

    return matched
    
# Convert to DataFrames
df1 = kicker
df2 = tf
df3 = fifa

# Merge using exact match on dateOfBirth
merged_df_fuzzy = pd.concat([df1, df2, df3], ignore_index=True)
date_to_compare = pd.to_datetime("09.07.1995", format="%d.%m.%Y")
merged_df_fuzzy = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"]==date_to_compare]
#print(merged_df_fuzzy)
grouped_players = []
checked = set()

# iterate through each player rows, player are from all three sources 
for i, row in merged_df_fuzzy.iterrows():
        #print(row.loc[["normalized_name", "lastName", "firstName"]].tolist())
        if row["id"] in checked:
            continue

        # group rows based on dob, current player as baseline
        potential_matches = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"] == row["dateOfBirth"]]  
        #print(f'{row["normalized_name"]}')
        #print(f'{potential_matches[["normalized_name"]]}')
        #matched = extract(row.loc[["normalized_name", "lastName", "firstName"]].tolist(), potential_matches[["normalized_name", "lastName", "firstName"]])
        matched = extract(row.loc[["normalized_name", "lastName"]].tolist(), potential_matches[["normalized_name", "lastName"]])
        print(f"Matched> {matched}")


            id   firstName            lastName     currentClub dateOfBirth  \
16    palhinha                   Joao Palhinha  Bayern München  1995-07-09   
597     257455        João            Palhinha   Bayern Munich  1995-07-09   
1014    229391  João Maria  Palhinha Gonçalves             NaN  1995-07-09   

      height  weight                       position     nations   age  ...  \
16     190.0    82.0                     Mittelfeld  [Portugal]  30.0  ...   
597    190.0     NaN  Midfield - Defensive Midfield         NaN   NaN  ...   
1014   190.0    83.0    Center Defensive Midfielder         NaN   NaN  ...   

            joined       expires  \
16             NaN           NaN   
597   Jul 11, 2024  Jun 30, 2028   
1014           NaN           NaN   

                                         marketValue   rank overallRating  \
16                                               NaN    NaN           NaN   
597   {'current': 40000000.0, 'highest': 60000000.0}    NaN           NaN  

In [293]:
# search terms is list of column names
# terms df holds the grouped players based on dob
def extract(search_terms:list, terms:pd.DataFrame):
    #print(f"{search_terms} :: {terms.to_dict()}")
    #print()
    
    matched = []
    for i, row in terms.iterrows():
        print(f'Row> {i}, {row} :: {type(row)} >>')
        for search_i, term in enumerate(search_terms):
            if row.iloc[search_i] == "" or search_terms[search_i] == "":
                matched.append((row.iloc[search_i], False, i))
                break
            print(f'Search> {row.iloc[search_i]} :: {search_terms[search_i]}')
            if (row.iloc[search_i] == search_terms[search_i]) or (row.iloc[search_i] in search_terms[search_i]) or (search_terms[search_i] in row.iloc[search_i]):
                matched.append((row.iloc[search_i], True, i))
                break
            else:
                matched.append((row.iloc[search_i], False, i))

    return matched
    
# Convert to DataFrames
df1 = kicker
df2 = tf
df3 = fifa

# Merge using exact match on dateOfBirth
merged_df_fuzzy = pd.concat([df1, df2, df3], ignore_index=True)
date_to_compare = pd.to_datetime("09.07.1995", format="%d.%m.%Y")
merged_df_fuzzy = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"]==date_to_compare]
#print(merged_df_fuzzy)
grouped_players = []
checked = set()

# iterate through each player rows, player are from all three sources 
for i, row in merged_df_fuzzy.iterrows():
        #print(row.loc[["normalized_name", "lastName", "firstName"]].tolist())
        if row["id"] in checked:
            continue

        # group rows based on dob, current player as baseline
        potential_matches = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"] == row["dateOfBirth"]]  
        print(potential_matches)
        #print(f'{row["normalized_name"]}')
        #print(f'{potential_matches[["normalized_name"]]}')
        #matched = extract(row.loc[["normalized_name", "lastName", "firstName"]].tolist(), potential_matches[["normalized_name", "lastName", "firstName"]])
        #matched = extract(row.loc[["normalized_name", "lastName"]].tolist(), potential_matches[["normalized_name", "lastName"]])
        #print(f"Matched> {matched}")


            id   firstName            lastName     currentClub dateOfBirth  \
16    palhinha                   Joao Palhinha  Bayern München  1995-07-09   
597     257455        João            Palhinha   Bayern Munich  1995-07-09   
1014    229391  João Maria  Palhinha Gonçalves             NaN  1995-07-09   

      height  weight                       position     nations   age  ...  \
16     190.0    82.0                     Mittelfeld  [Portugal]  30.0  ...   
597    190.0     NaN  Midfield - Defensive Midfield         NaN   NaN  ...   
1014   190.0    83.0    Center Defensive Midfielder         NaN   NaN  ...   

            joined       expires  \
16             NaN           NaN   
597   Jul 11, 2024  Jun 30, 2028   
1014           NaN           NaN   

                                         marketValue   rank overallRating  \
16                                               NaN    NaN           NaN   
597   {'current': 40000000.0, 'highest': 60000000.0}    NaN           NaN  

In [309]:
#names = ["Palhinha", "Joao Palhinha", "Palhinha Gonçalves"]
#names = ["João", "", "João Maria"]
#names = ["leo_scienza", "leonardo_scienza", "leonardo_scienza"]
names = ["alejandro_grimaldo","eric_oelschlagel","alejandro_grimaldo","eric_oelschlagel","alejandro_grimaldo_garcia"]

for i in range(len(names)):
    for j in range(i + 1, len(names)):  # Avoid self-comparison and duplicate pairs
        print(f"{names[i]} == {names[j]} → {names[i] == names[j]}")
        print(f"{names[i]} in {names[j]} → {names[i] in names[j]} or {names[j] in names[i]}")
        print()


alejandro_grimaldo == eric_oelschlagel → False
alejandro_grimaldo in eric_oelschlagel → False or False

alejandro_grimaldo == alejandro_grimaldo → True
alejandro_grimaldo in alejandro_grimaldo → True or True

alejandro_grimaldo == eric_oelschlagel → False
alejandro_grimaldo in eric_oelschlagel → False or False

alejandro_grimaldo == alejandro_grimaldo_garcia → False
alejandro_grimaldo in alejandro_grimaldo_garcia → True or False

eric_oelschlagel == alejandro_grimaldo → False
eric_oelschlagel in alejandro_grimaldo → False or False

eric_oelschlagel == eric_oelschlagel → True
eric_oelschlagel in eric_oelschlagel → True or True

eric_oelschlagel == alejandro_grimaldo_garcia → False
eric_oelschlagel in alejandro_grimaldo_garcia → False or False

alejandro_grimaldo == eric_oelschlagel → False
alejandro_grimaldo in eric_oelschlagel → False or False

alejandro_grimaldo == alejandro_grimaldo_garcia → False
alejandro_grimaldo in alejandro_grimaldo_garcia → True or False

eric_oelschlagel == al

In [335]:
# search terms is list of column names
# terms df holds the grouped players based on dob
def extract(search_terms:list, terms:pd.DataFrame):
    #print(f"{search_terms} :: {terms.to_dict()}")
    #print()
    
    matched = []
    for i, row in terms.iterrows():
        #print(f'Row> {i}, {row} :: {type(row)} >>')
        for search_i, term in enumerate(search_terms):
            if row.iloc[search_i] == "" or search_terms[search_i] == "":
                matched.append((row.iloc[search_i], False, i))
                break
            #print(f'Search> {row.iloc[search_i]} :: {search_terms[search_i]}')
            if (row.iloc[search_i] == search_terms[search_i]) or (row.iloc[search_i] in search_terms[search_i]) or (search_terms[search_i] in row.iloc[search_i]):
                matched.append((row.iloc[search_i], True, i))
                break
            else:
                matched.append((row.iloc[search_i], False, i))

    return matched
    
# Convert to DataFrames
df1 = kicker
df2 = tf
df3 = fifa

# Merge using exact match on dateOfBirth
merged_df_fuzzy = pd.concat([df1, df2, df3], ignore_index=True)
date_to_compare = pd.to_datetime("25.09.1995", format="%d.%m.%Y")
merged_df_fuzzy = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"]==date_to_compare]
#print(merged_df_fuzzy)
grouped_players = []
checked = set()

for i, row in merged_df_fuzzy.iterrows():
    if row["id"] in checked:
        continue

    potential_matches = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"] == row["dateOfBirth"]]
    #print(f'{row[["id", "normalized_name", "dateOfBirth"]]} :: {potential_matches[["id", "normalized_name"]]}')

    matched = extract(row.loc[["normalized_name"]].tolist(), potential_matches[["normalized_name"]])
    print(matched)
    match_ids = []
    for match_name, score, idx in matched:
        
        if score:
            match_ids.append(merged_df_fuzzy.loc[idx]["id"])
            checked.add(merged_df_fuzzy.loc[idx]["id"])

    grouped_players.append({
        "dateOfBirth": row["dateOfBirth"],
        "matched_ids": match_ids
     })

    
print(pd.DataFrame(grouped_players))


[('adam_dzwigala', True, 395), ('adam_dzwigala', True, 751), ('adam_dzwigała', False, 1378)]
[('adam_dzwigala', False, 395), ('adam_dzwigala', False, 751), ('adam_dzwigała', True, 1378)]
  dateOfBirth              matched_ids
0  1995-09-25  [adam-dzwigala, 236966]
1  1995-09-25                 [211362]


In [406]:
def transform_tuples(input_list):
    # Define the desired order of sources
    source_order = ['kicker', 'tf', 'fifa']
    
    # Convert list of tuples into a dictionary
    #source_dict = dict(input_list)
    source_dict = dict(map(lambda x: (x[1], x[0]), input_list))
    print(source_dict)    
    # Create a list of values following the defined order, using "" if missing
    transformed_list = [source_dict.get(source, "") for source in source_order]
    
    return transformed_list

def deep_check(n1, n2):
    names=[n1,n2]

    names_list=[name.split("_") for name in names]
    
    # Convert each sublist to a set
    sets = [set(sublist) for sublist in names_list]
    print(sets)
    
    # Find the intersection (common elements in all sets)
    common_names = set.intersection(*sets)
    #print(common_names)
    # Check if there is at least one common name
    result = bool(common_names)
    
    #print(result)  # Output: True
    return result

# search terms is list of column names
# terms df holds the grouped players based on dob
def extract(search_terms:list, terms:pd.DataFrame, columns:list):
    #print(f"{search_terms} :: {terms.to_dict()}")
    #print()
    search_terms=search_terms.loc[columns].tolist()
    terms=terms[columns]
    
    matched = []
    for i, row in terms.iterrows():
        #print(row.index)
        #print(f'Row> {i}, {row} :: {type(row)} >>')
        for search_i, term in enumerate(search_terms):
            if row.iloc[search_i] == "" or search_terms[search_i] == "":
                matched.append((row.iloc[search_i], False, i, row["source"]))
                break
            #print(f'Search> {row.iloc[search_i]} :: {search_terms[search_i]}')
            if (row.iloc[search_i] == search_terms[search_i] 
               or row.iloc[search_i] in search_terms[search_i] 
               or search_terms[search_i] in row.iloc[search_i]
                or deep_check(search_terms[search_i], row.iloc[search_i])
               ):
                matched.append((row.iloc[search_i], True, i, row["source"]))
                break
            else:
                matched.append((row.iloc[search_i], False, i, row["source"]))
                break

    return matched
    
# Convert to DataFrames
df1 = kicker
df2 = tf
df3 = fifa

# Merge using exact match on dateOfBirth
merged_df_fuzzy = pd.concat([df1, df2, df3], ignore_index=True)
#date_to_compare = pd.to_datetime("03.06.2002", format="%d.%m.%Y")
#merged_df_fuzzy = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"]==date_to_compare]
#print(merged_df_fuzzy)
grouped_players = []
checked = set()

for i, row in merged_df_fuzzy.iterrows():
    if row["id"] in checked:
        continue

    potential_matches = merged_df_fuzzy[merged_df_fuzzy["dateOfBirth"] == row["dateOfBirth"]]
    print(f'>>{row[["id", "normalized_name", "source"]]} :: {potential_matches[["id", "normalized_name", "source"]]}<<')
    #print(potential_matches[["lastName", "source"]])
    matched = extract(
        row, 
        potential_matches,
        ["normalized_name", "lastName", "firstName", "source"]
    )
    
    print(f'{row.loc[["normalized_name", "lastName"]].tolist()} :: {matched}')
    print()

    match_ids = []
    for match_name, score, idx, source in matched: 
        if score:
            match_ids.append((merged_df_fuzzy.loc[idx]["id"], merged_df_fuzzy.loc[idx]["source"]))
            checked.add(merged_df_fuzzy.loc[idx]["id"])

    grouped_players.append({
        "dateOfBirth": row["dateOfBirth"],
        "matched_ids": transform_tuples(match_ids)
     })

all=pd.DataFrame(grouped_players)
print(pd.DataFrame(grouped_players))
all.to_csv("D:\\DevOps\\python_work\\venv\\demoenv\\resources\\all.csv", encoding='utf-8', index=False)

>>id                 manuel-neuer
normalized_name    manuel_neuer
source                   kicker
Name: 0, dtype: object ::                 id normalized_name  source
0     manuel-neuer    manuel_neuer  kicker
582          17259    manuel_neuer      tf
1011        167495    manuel_neuer    fifa<<
['manuel_neuer', 'Neuer'] :: [('manuel_neuer', True, 0, 'kicker'), ('manuel_neuer', True, 582, 'tf'), ('manuel_neuer', True, 1011, 'fifa')]

{'kicker': 'manuel-neuer', 'tf': 17259, 'fifa': 167495}
>>id                 daniel-peretz
normalized_name    daniel_peretz
source                    kicker
Name: 1, dtype: object ::                  id normalized_name  source
1     daniel-peretz   daniel_peretz  kicker
583          468539   daniel_peretz      tf
1296         277823   daniel_peretz    fifa<<
['daniel_peretz', 'Peretz'] :: [('daniel_peretz', True, 1, 'kicker'), ('daniel_peretz', True, 583, 'tf'), ('daniel_peretz', True, 1296, 'fifa')]

{'kicker': 'daniel-peretz', 'tf': 468539, 'fifa': 2778

In [361]:
# Convert to DataFrames
df1 = kicker
df2 = tf
df3 = fifa

# Merge using exact match on dateOfBirth
merged_df_fuzzy = pd.concat([df1, df2, df3], ignore_index=True)

print(merged_df_fuzzy[["id", "dateOfBirth", "normalized_name"]].sort_values(by=['dateOfBirth'], ascending=False))
n1="joao_palhinha"
n2="joao_palhinha"
n3="joao_maria_palhinha_goncalves"

names=[n1,n2,n3]

names_list=[name.split("_") for name in names]

# Convert each sublist to a set
sets = [set(sublist) for sublist in names_list]
print(sets)

# Find the intersection (common elements in all sets)
common_names = set.intersection(*sets)
print(common_names)
# Check if there is at least one common name
result = bool(common_names)

print(result)  # Output: True



                   id dateOfBirth  normalized_name
158        faik-sakar  2008-01-14       faik_sakar
144       viggo-gebel  2007-11-22      viggo_gebel
49     artem-stepanov  2007-08-10   artem_stepanov
981           1075579  2007-04-21    mathys_angely
164     mathys-angely  2007-04-21    mathys_angely
...               ...         ...              ...
582             17259  1986-03-27     manuel_neuer
0        manuel-neuer  1986-03-27     manuel_neuer
960            111266  1986-03-10  anthony_losilla
440   anthony-losilla  1986-03-10  anthony_losilla
1207           199288  1986-03-10  anthony_losilla

[1486 rows x 3 columns]
[{'joao', 'palhinha'}, {'joao', 'palhinha'}, {'goncalves', 'maria', 'joao', 'palhinha'}]
{'joao', 'palhinha'}
True


In [ ]:
def filter_missing_fifa(normalized_data:list):
    #print(normalized_data['matched_ids'].head().tolist())
    """Return only tuples where FIFA is empty."""
    #l=[item for item in normalized_data['matched_ids'].tolist()]
    #print(f"{type(l[1])} :: {l}")
    l=[item for item in normalized_data if item[2] == ""]
    return l


def transform_tuples(input_list):
    # Define the desired order of sources
    source_order = ['kicker', 'tf', 'fifa']
    
    # Convert list of tuples into a dictionary
    #source_dict = dict(input_list)
    source_dict = dict(map(lambda x: (x[1], x[0]), input_list))
    print(source_dict)    
    # Create a list of values following the defined order, using "" if missing
    transformed_list = [source_dict.get(source, "") for source in source_order]
    
    return transformed_list

# Example lists of tuples
list1 = [('yan-couto', 'kicker'), (627228, 'tf'), (259075, 'fifa')]
list2 = [(262665, 'fifa')]

# Transform the lists 
result1 = transform_tuples(list1)
result2 = transform_tuples(list2)

# Print the results
print(result1)  # Output: ['yan-couto', 627228, 259075]
print(result2)  # Output: ['', '', 262665]


In [463]:
import ast

PLAYERS_ALL_FILE='D:\\DevOps\\python_work\\venv\\demoenv\\resources\\all.csv'
players_all=pd.read_csv(PLAYERS_ALL_FILE)
players_all.head()
print(players_all.head())
players_list=players_all["matched_ids"].apply(ast.literal_eval)
print(f"{players_list[:5]} :: {type(players_list.tolist())}")
print(players_list.tolist()[:5])
print(filter_missing_fifa(players_list.tolist()))
wo_fifa=filter_missing_fifa(players_list.tolist())

[(sublist[0], 'kicker') if sublist[0] else (sublist[1], 'tf') for sublist in wo_fifa]

  dateOfBirth                        matched_ids
0  1986-03-27    ['manuel-neuer', 17259, 167495]
1  2000-07-10  ['daniel-peretz', 468539, 277823]
2  1988-08-03    ['sven-ulreich', 40680, 186569]
3  2003-08-08        ['jonas-urbig', 607720, '']
4  2000-09-13     ['sacha-boey', 475413, 248266]
0      [manuel-neuer, 17259, 167495]
1    [daniel-peretz, 468539, 277823]
2      [sven-ulreich, 40680, 186569]
3            [jonas-urbig, 607720, ]
4       [sacha-boey, 475413, 248266]
Name: matched_ids, dtype: object :: <class 'list'>
[['manuel-neuer', 17259, 167495], ['daniel-peretz', 468539, 277823], ['sven-ulreich', 40680, 186569], ['jonas-urbig', 607720, ''], ['sacha-boey', 475413, 248266]]
[['jonas-urbig', 607720, ''], ['arthur-5', 977464, ''], ['mario-hermoso', 281769, ''], ['emiliano-buendia', 321247, ''], ['alejo-sarco', 1000675, ''], ['artem-stepanov', '', ''], ['tuta', 546213, ''], ['michy-batshuayi', 179184, ''], ['sepe-elye-wahi', 659542, ''], ['maxim-dal', '', ''], ['maxim-leitsch', 

[('jonas-urbig', 'kicker'),
 ('arthur-5', 'kicker'),
 ('mario-hermoso', 'kicker'),
 ('emiliano-buendia', 'kicker'),
 ('alejo-sarco', 'kicker'),
 ('artem-stepanov', 'kicker'),
 ('tuta', 'kicker'),
 ('michy-batshuayi', 'kicker'),
 ('sepe-elye-wahi', 'kicker'),
 ('maxim-dal', 'kicker'),
 ('maxim-leitsch', 'kicker'),
 ('arnaud-nordin', 'kicker'),
 ('bruno-ogbus', 'kicker'),
 ('jan-niklas-beste', 'kicker'),
 ('kosta-nedeljkovic', 'kicker'),
 ('viggo-gebel', 'kicker'),
 ('joyeux-masanka-bungi', 'kicker'),
 ('tidiam-gomis', 'kicker'),
 ('faik-sakar', 'kicker'),
 ('mathys-angely', 'kicker'),
 ('mads-roerslev-rasmussen', 'kicker'),
 ('andreas-skov-olsen', 'kicker'),
 ('niklas-swider', 'kicker'),
 ('shio-fukuda', 'kicker'),
 ('tiago-pereira-cardoso', 'kicker'),
 ('anrie-chase', 'kicker'),
 ('luca-jaquez', 'kicker'),
 ('finn-jeltsch', 'kicker'),
 ('silas-ostrzinski', 'kicker'),
 ('almugera-kabar', 'kicker'),
 ('filippo-calixte-mane', 'kicker'),
 ('daniel-svensson', 'kicker'),
 ('carney-chukwuemek

In [465]:
kicker["id"].get("jonas-urbig", "N/A")

'N/A'